Operartis-1A KNN Algorithm

In [1]:
import pandas as pd
import numpy as np
import os 

In [2]:
filename = os.path.join(os.getcwd(), "train.csv")
df = pd.read_csv(filename, header=0)

In [3]:
df.head()

,matchId,matchDate,matchRule,matchedBy,wasPreviouslyMismatched,generatorArtifactName,generatorId,generatorImportDate,A_debitOrCredit,A_amount,...,B_amount,B_valueDate,B_currencyCode,B_account,B_transactionReferences,B_transactionAttributes,B_orderingPartyInfo,B_receivingPartyInfo,targetAllocation,generatorAllocation
0,1287,9/18/13,MANUAL,Jane Doe,0,A,1291.0,9/18/13,NONE,5481.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULT,USD_9/11/13_3446886485_600 cc AAB
1,1293,9/18/13,MANUAL,Jane Doe,0,A,1313.0,9/18/13,NONE,"34,233.97",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULT,USD_9/17/13_3445996130_211 AAB
2,1293,9/18/13,MANUAL,Jane Doe,0,A,1314.0,9/18/13,NONE,"32,643.27",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULT,USD_9/17/13_3445996130_209 AAB
3,1293,9/18/13,MANUAL,Jane Doe,0,A,1315.0,9/18/13,NONE,"36,897.54",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULT,USD_9/17/13_3445996130_205 AAB
4,1293,9/18/13,MANUAL,Jane Doe,0,A,1316.0,9/18/13,NONE,"31,444.64",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULT,USD_9/17/13_3445996130_202 AAB
